In [ ]:
from src.data import Dataset, available_datasets
from src import workflow

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger()

In [ ]:
dataset_list = workflow.available_datasets()
dataset_list

## Overall task: 

Train a supervised model on the lvq-pak. Try three different techniques, three times, and pick the one with the best accuracy score.

## Get data

In [ ]:
ds_test = Dataset.load('lvq-pak_test')

In [ ]:
ds_test.data.shape

In [ ]:
ds_train = Dataset.load('lvq-pak_train')

In [ ]:
ds_train.data.shape

In [ ]:
ds_train.target

In [ ]:
print(ds_train.DESCR)

## Train an algorithm

Let's start with one algorithm!

`make train`

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
model = LinearSVC(random_state=42)

In [ ]:
model.fit(ds_train.data, ds_train.target)

In [ ]:
%%time
model = LinearSVC(random_state=42, max_iter=200000)
model.fit(ds_train.data, ds_train.target)

## Use it to predict
`make predict`

In [ ]:
p_test = model.predict(ds_test.data);
p_test

## Test the quality of the prediction
`make analysis`

In [ ]:
model.score(ds_test.data, ds_test.target)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(ds_test.target, p_test)

## Next: 
* automate the basic workflow
* compare 3 different algorithms run with 3 different random states for our Swedish Chef paper

# Step 1: `make train`

## Add our algorithm to available_algorithms

There are currently no available algorithms.

In [ ]:
workflow.available_algorithms()

To add an algorithm, add  a key:value pair to the dict `_ALGORITHMS` in `src/models/algorithms.py`.

For example, add
```
'linearSVC': LinearSVC()
```
to the `_ALGORITHMS` dict, and add
```
from sklearn.svm import LinearSVC
```
to the top of the file.

Also, add `linearSVC` to the docstring of `available_algorithms`.

In [ ]:
workflow.available_algorithms()

In [ ]:
help(workflow.available_algorithms)

Now we're in a position where the `make train` script can run using `linearSVC`. 

In [ ]:
!cd .. && make -n train

You'll notice that `make train` takes a `models/model_list.json` as input.
```
## train / fit / build models
train: models/model_list.json
	$(PYTHON_INTERPRETER) -m src.models.train_model model_list.json
```

Under the hood, a `model_list.json` is a list of dicts, where each dict specifices a combination of:
* `dataset_name`: A valid dataset name from `available_datasets`
* `algorithm_name`: A valid dataset name from `available_algorithms`
* `algorithm_params`: A dictionary of parameters to use when running the specified `algorithm`
* `run_number`: (optional, default 1) A unique integer used to distinguish between different builds with otherwise identical parameters

We don't need to know this, as we will use helper functions in `workflow` to build it.

In [ ]:
workflow.add_model(dataset_name='lvq-pak_train',
                   algorithm_name="linearSVC",
                   algorithm_params={'random_state': 42, 'max_iter': 200000})

In [ ]:
workflow.get_model_list()

In [ ]:
!cat ../models/model_list.json

Now running `make train` will train `LinearSVC` on `lvq-pak` with the specified parameters.

The output will be:
* A trained model in `models/trained_models`
* A json file `models/trained_models.json` that keeps track of the models that we've trained

In [ ]:
workflow.build_models()

In [ ]:
!cd .. && make train

## TODO: Caching! Then, checking against existing files and metadata and looking for caching! (note: will need a force parameter eventually)

## TODO: Don't overwrite the trained_models.json, append to it (as long as the files are still there)

### Let's take a look at the output from `make train`

In [ ]:
from src.paths import trained_model_path
from src.utils import list_dir
from src.utils import load_json

In [ ]:
workflow.available_trained_models()

In [ ]:
list_dir(trained_model_path)

In [ ]:
# load up the trained model
from src.models.train import load_model

s_model, s_model_metadata = load_model(model_name='linearSVC_lvq-pak_train_0', model_path=trained_model_path)

In [ ]:
s_model

In [ ]:
s_model_metadata

In [ ]:
ds = Dataset.load('lvq-pak_train')
ds.DATA_HASH

## TODO: explore the effects of caching once it's implemented

Any algorithm will work that: 
* is a subclass of the sklearn `BaseEstimator` class (needed for setting and getting params)
* has a `fit` method (needed for `make train`)
* has either a `predict` method (supervised) or a `transform` method (unsupervised) (needed for `make predict`)

We will see how things work in the unsupervised case in the next example. 

Note that an **algorithm** here can be a combination of "algorithms" as long as that combination is a `BaseEstimator` with the above methods. For example, you can use an sklearn pipeline, or an sklearn meta estimator like GridSearchCV as an algorithm. 

If your algorithm of choice is **not yet** a `BaseEstimator` with the appropriate API, it is fairly easy to wrap it to be used in this way. While we won't have time to cover an example of this during the in-person part of this tutorial, the EDA Text Embedding (advanced usage tutorial) has an example of how to do this with gensim's FastText model.



# Step 2: `make predict`

```
## predict / transform / run experiments
predict: models/predict_list.json
	$(PYTHON_INTERPRETER) -m src.models.predict_model predict_list.json
```

Similar to `models_list.json` in `predict_list.json` we specify the dataset to operate on, and in this case, the trained_model to apply to the given dataset.


A `predict_list.json` is a list of dicts, where each dict specifices a combination of:
* `dataset_name`: A valid dataset name from `available_datasets`
* `dataset_params`: A dictionary of parameters that can be passed to `load_dataset()` with the specified `dataset`
* `model_name`: A valid dataset name from `available_trained_models` (aka. a key name in `trained_models.json`
* `is_supervised`: Whether to use the `predict` (supervised) or `transform` (unsupervised) method


Let's use the test set here to do the prediction.

In [ ]:
workflow.add_prediction(dataset_name='lvq-pak_test', model_name='linearSVC_lvq-pak_train_0', is_supervised=True)

```
predict_list = [
    {
        'dataset_name': 'lvq-pak_test',
        'model_name': 'linearSVC_lvq-pak_train_0',
        'is_supervised': True
    }
]
```

In [ ]:
workflow.get_prediction_list()

In [ ]:
!cd .. && LOGLEVEL=INFO make predict

## TODO don't overwrite predictions.json

In [ ]:
reload(workflow)
workflow.available_predictions()

In [ ]:
from src.paths import model_output_path

Remember: Predictions are just Datasets tagged with experiment data

In [ ]:

predict_ds = Dataset.load('linearSVC_lvq-pak_train_0_exp_lvq-pak_test_0', data_path=model_output_path)

In [ ]:
predict_ds.metadata['experiment']

In [ ]:
predict_ds.data.shape

## TODO: trained/tested on different data so far...this should work once the lvq-pak stuff is changed

In [ ]:
all(predict_ds.data == p_test)

# Step 3: Analysis

## TODO: Move this function to a module

In [ ]:
from src.data import Dataset
from src.paths import model_output_path

In [ ]:
import pandas as pd

In [ ]:
def available_scorers():
    _SCORERS = {
        'accuracy_score': accuracy_score
    }
    return _SCORERS

In [ ]:
def supervised_score_df(predict_json='predictions.json', predict_json_path=None,
                        score_list=['accuracy_score']):
    if predict_json_path is None:
        predict_json_path = model_path
    else:
        predict_json_path = pathlib.Path(predict_json_path)
    predictions = load_json(predict_json_path / predict_json)
    score_df = pd.DataFrame(columns=['score_name', 'algorithm_name', 'dataset_name',
                                     'model_name', 'model_run_number'])
    for current_scorer_name in score_list:
        current_scorer = available_scorers()[current_scorer_name]

        score_dict = {}
        score_dict['score_name'] = current_scorer_name
        for key in predictions.keys():
            prediction = predictions[key]
            exp = prediction['experiment']
            pred_ds = load_prediction(prediction['dataset_name'], predict_path=model_output_path)

            ds_name = exp['dataset_name']
            ds = Dataset.load(ds_name)
            
            ds_name = ds_name.rsplit('_test')[0]
            ds_name = ds_name.rsplit('_train')[0]
            score_dict['dataset_name'] = ds_name

            score_dict['score'] = current_scorer(ds.target, pred_ds.data)

            model_metadata = load_model(model_name=exp['model_name'], metadata_only=True)
            score_dict['algorithm_name'] = model_metadata['algorithm_name']
            score_dict['model_name'] = exp['model_name']
            score_dict['model_run_number'] = exp['run_number']
            new_score_df = pd.DataFrame(score_dict, index=[0])
            score_df = score_df.append(new_score_df, sort=True)
    return score_df

In [ ]:
supervised_score_df()

## TODO: Add caching of the summary dfs to know if you're about to overwrite one

# Step 4: Add other algorithms


### Exercise: Add GradientBoostingClassifier and some other sklearn Classifier of your choice

### Advanced Exercise: Use GridSearchCV applied to your classifier of choice as the 3rd alg

In [ ]:
model_list = [
    {
        'dataset_name': 'lvq-pak_train',
        'algorithm_name': 'linearSVC',
        'algorithm_params': {'random_state': 42, 'max_iter': 200000},
    },
    {
        'dataset_name': 'lvq-pak_train',
        'algorithm_name': 'GradientBoostingClassifier',
        'algorithm_params': {'random_state': 42},
    },
    {
        'dataset_name': 'lvq-pak_train',
        'algorithm_name': 'GridSearchCV',
        'algorithm_params': {'alg_name': 'RandomForestClassifier',
                             'alg_params': {'n_estimators': 200},
                             'gridsearch_params':{'max_features':['sqrt', 'log2', 10],
                                                   'max_depth':[5, 7, 9],
                                                   'random_state':[42, 62345, 3457],
                                                   },
                             'params': {'cv': 3}
                            }
    }
]

save_json(model_path / 'model_list.json', model_list)

In [ ]:
!cd .. && make train

In [ ]:
load_json(model_path / 'trained_models.json')

## TODO: Figure out where and how to include a "lesson" on random_state
